In [ ]:
import rasterio
import rasterio.plot
import pyproj
import matplotlib
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import osr
import math
from rasterio.plot import show
import fiona
import rasterio
import rasterio.mask
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
import os
import re
import statistics

In [ ]:
folder=r'//'#Location of CHIRPS Dataset Tiff files
print(os.listdir(folder))

In [ ]:
# Renaming Tiff files for convienience
count_year=1981
count_month=1
for file_name in os.listdir(folder):
    if count_month==13:
        count_month=1
        count_year+=1
    source=folder+file_name
    if count_month<10:
        month="0"+str(count_month)
    else:
        month=str(count_month)
    destination=folder+str(count_year)+"_"+month+".tif"
    os.rename(source,destination)
    count_month+=1
print(os.listdir(folder))

In [ ]:
#Changing directory to load files from and save output in
os.chdir(r'//')#Location of CHIRPS Dataset Tiff files

In [ ]:
#Cropping Dataset to ROI using shp file
shp_file_path = r'//'#Location of Shapefile
with fiona.open(shp_file_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
src_raster_path = r'//'#Location of Rasters
folder=r'//'#Location of Rasters(For saving clipped rasters)
for file_name in os.listdir(folder):
    src_raster_path_name=src_raster_path+file_name
    output_raster_path = src_raster_path_name[:-4] + '_water_masked.tif'
    with rasterio.open(src_raster_path_name) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta
        height=out_image.shape[1]
        width=out_image.shape[2]
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    with rasterio.open(output_raster_path, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
#Changing directory to load files from and save output in
os.chdir(r'//')#Location of Karnataka Dataset Tiff files
#Checking if gdal library is loaded properly- sometimes it doesnt load properly
!gdalinfo -nomd 1981_01_water_masked.tif

In [ ]:
folder=r'//'#Location of Karnataka Dataset Tiff files
print(os.listdir(folder))

In [ ]:
#making a list of all raster files
rasternames=[]
for file_name in os.listdir(folder):
    rasternames.append(file_name)

In [ ]:
#load one image to get various values needed later for creating rasters and mask
img1=gdal.Open('//')#Location of Raster
col=img1.RasterXSize
rows=img1.RasterYSize
driver=img1.GetDriver()
img1_val=img1.GetRasterBand(1).ReadAsArray().flatten()

In [ ]:
#mask creation to remove values outside of boundary in loaded tiff files values later
mask_in=[]
for i in range(0,len(img1_val)):
    if img1_val[i]!=0:# replace with default value outside of boundary
        mask_in.append(i)

In [ ]:
# Making lists to store each months tiff files seperately
jan=[]
feb=[]
mar=[]
apr=[]
may=[]
jun=[]
jul=[]
aug=[]
sep=[]
octo=[]
nov=[]
dec=[]
for i in rasternames:
    if re.search("_01_",i):
        jan.append(i)
    if re.search("_02_",i):
        feb.append(i)
    if re.search("_03_",i):
        mar.append(i)
    if re.search("_04_",i):
        apr.append(i)
    if re.search("_05_",i):
        may.append(i)
    if re.search("_06_",i):
        jun.append(i)
    if re.search("_07_",i):
        jul.append(i)
    if re.search("_08_",i):
        aug.append(i)
    if re.search("_09_",i):
        sep.append(i)
    if re.search("_10_",i):
        octo.append(i)
    if re.search("_11_",i):
        nov.append(i)
    if re.search("_12_",i):
        dec.append(i)
#Storing all lists in one list to make access easy
month=[jan,feb,mar,apr,may,jun,jul,aug,sep,octo,nov,dec]

In [ ]:
#helps create a empty list for result raster creation
def zero(list):
    for i in range(0,len(list)):
        list[i]=0
    return list

In [ ]:
# Average of each month
i=0
for m in month:
    avg_val=zero(img1_val)
    for y in m:
        img=gdal.Open(y)
        img_val=img.GetRasterBand(1).ReadAsArray().flatten()
        avg_val=avg_val+img_val
    avg_val=avg_val/40
    avgdataMatrix=avg_val.reshape(rows,col)
    if i<9:
        raster_avg = driver.Create("average_of_0"+str(i+1)+ ".tif", col, rows, 1, gdal.GDT_Float32)
    else:
        raster_avg = driver.Create("average_of_"+str(i+1)+ ".tif", col, rows, 1, gdal.GDT_Float32)
    raster_avg.SetGeoTransform(img1.GetGeoTransform())
    raster_avg.SetProjection(img1.GetProjection())
    raster_avg.GetRasterBand(1).WriteArray(avgdataMatrix)
    i+=1
    del raster_avg

In [ ]:
#saving all averages in one list for convenience
avg=[]
for i in os.listdir(folder):
    if re.search("average_of",i):
        avg.append(i)

# Standard Deviation for each month

In [ ]:
# Standard Deviation for Each month
for m in range(0,12):
    avg_img=gdal.Open(avg[m])
    avg_val=avg_img.GetRasterBand(1).ReadAsArray().flatten()
    diff=zero(img1_val)
    diff_sum=zero(img1_val)
    for y in range(0,len(month[m])):
        img=gdal.Open(month[m][y])
        img_val=img.GetRasterBand(1).ReadAsArray().flatten()
        diff=avg_val-img_val
        diff=numpy.square(diff)
        diff_sum=diff_sum+diff
    diff_sum=diff_sum/40
    sd=numpy.sqrt(diff_sum)
    sdMatrix=sd.reshape(rows,col)
    if m<9:
        raster_sd = driver.Create("sd_of_0" + str(m+1) + ".tif", col, rows, 1, gdal.GDT_Float32)
    else:
        raster_sd = driver.Create("sd_of_" + str(m+1) + ".tif", col, rows, 1, gdal.GDT_Float32)
    raster_sd.SetGeoTransform(img1.GetGeoTransform())
    raster_sd.SetProjection(img1.GetProjection())
    raster_sd.GetRasterBand(1).WriteArray(sdMatrix)
    del raster_sd

In [ ]:
#saving all standard deviations in one list for convenience
sd=[]
for i in os.listdir(folder):
    if re.search("sd_of",i):
        sd.append(i)

# Anomaly and Standardized Anomaly

In [ ]:
# Anomaly and Standardized Anomaly for all
i=0
for m in month:
    avg_img=gdal.Open(avg[i])
    avg_val=avg_img.GetRasterBand(1).ReadAsArray().flatten()
    sd_img=gdal.Open(sd[i])
    sd_val=sd_img.GetRasterBand(1).ReadAsArray().flatten()
    i+=1
    for y in m:
        img=gdal.Open(y)
        img_val=img.GetRasterBand(1).ReadAsArray().flatten()
        anomaly=img_val-avg_val
        sdt_anomaly=numpy.divide(anomaly,sd_val,out=numpy.zeros_like(anomaly), where=sd_val!=0)
        anomalyMatrix=anomaly.reshape(rows,col)
        sdt_anomalyMatrix=sdt_anomaly.reshape(rows,col)
        raster_anomaly=driver.Create("anomaly_" + y[0:7] + ".tif",col,rows,1,gdal.GDT_Float32)
        raster_anomaly.SetGeoTransform(img1.GetGeoTransform())
        raster_anomaly.SetProjection(img1.GetProjection())
        raster_anomaly.GetRasterBand(1).WriteArray(anomalyMatrix)
        raster_sdt_anomaly=driver.Create("sdt_anomaly_" + y[0:7] + ".tif",col,rows,1,gdal.GDT_Float32)
        raster_sdt_anomaly.SetGeoTransform(img1.GetGeoTransform())
        raster_sdt_anomaly.SetProjection(img1.GetProjection())
        raster_sdt_anomaly.GetRasterBand(1).WriteArray(sdt_anomalyMatrix)
        del raster_anomaly
        del raster_sdt_anomaly

In [ ]:
#saving all standardized anomaly in one list for convenience
sdt_anml=[]
for i in rasternames:
    if re.search("sdt_anomaly",i):
        sdt_anml.append(i)

# Percentage Deviation From Mean

In [ ]:
i=0
for m in month:
    avg_img=gdal.Open(avg[i])
    avg_val=avg_img.GetRasterBand(1).ReadAsArray().flatten()
    i+=1
    for y in m:
        img=gdal.Open(y)
        img_val=img.GetRasterBand(1).ReadAsArray().flatten()
        anomaly=img_val-avg_val
        perc_anomaly=numpy.divide(anomaly,avg_val,out=numpy.zeros_like(anomaly), where=avg_val!=0)
        perc_anomaly=perc_anomaly*100
        perc_anomalyMatrix=perc_anomaly.reshape(rows,col)
        raster_perc_anomaly=driver.Create("percentage_anomaly_" + y[0:7] + ".tif",col,rows,1,gdal.GDT_Float32)
        raster_perc_anomaly.SetGeoTransform(img1.GetGeoTransform())
        raster_perc_anomaly.SetProjection(img1.GetProjection())
        raster_perc_anomaly.GetRasterBand(1).WriteArray(perc_anomalyMatrix)
        del raster_perc_anomaly

In [ ]:
#saving all percentage deviation in one list for convenience
perc_anml=[]
for i in rasternames:
    if re.search("percentage_anomaly",i):
        perc_anml.append(i)

# Monitoring

# Functions needed in monitoring

In [ ]:
def masking_3(arr):
    return [value for value in arr if value != 0]

In [ ]:
def classification_1(abc):
    reslt=[]
    for i in range(0,len(abc)):
        if abc[i]<0:#change value based on drought severity
            reslt.append(0)
        else:
            reslt.append(1)
    return reslt

In [ ]:
def rle_2(val_list):
    rle_output=[]
    i=0
    while (i <= len(val_list)-1):
        count=1
        ch=val_list[i]
        j=i
        while (j < len(val_list)-1):
            if (val_list[j] == val_list[j+1]):
                count=count+1
                j=j+1
            else:
                break
        rle_output.append(ch)
        rle_output.append(count)
        i=j+1
    return rle_output

# Actual Monitoring start 

In [ ]:
#Open and load rasters for distribution that needs to be monitored that need to be 
sdt_anml_val_3=[]
for i in range (0,len(sdt_anml)):
    img=gdal.Open(sdt_anml[i])
    val=img.GetRasterBand(1).ReadAsArray().flatten()
    val=masking_3(val)
    val=classification_1(val)
    sdt_anml_val_3.append(val)
print(len(sdt_anml_val_3))

# Monitoring phase 1- Frequency, Mode Onset/Cessation, Duration(Mean,Min,Max)

In [ ]:
#function for frequency, duration onset and cessation
def monitoring(abc):
    result=[]
    freq=0
    month_count=0
    start_year=[]
    start_month=[]
    end_year=[]
    end_month=[]
    duration=[]
    for k in range(0,len(abc),2):
        if abc[k]==0 and abc[k+1]>=3:
            duration.append(abc[k+1])
            freq+=1
            sy=month_count//12
            sm=month_count%12
            sm=sm+1
            start_year.append(sy)
            start_month.append(sm)
            month_count=month_count+abc[k+1]
            em=sm+abc[k+1]-2
            em=em%12
            em=em+1
            end_month.append(em)
            ey=month_count//12
            end_year.append(ey)
        else:
            month_count=month_count+abc[k+1]
    result.append(freq)
    result.append(statistics.mode(start_month))
    result.append(statistics.mode(end_month))
    result.append(statistics.mean(duration))
    result.append(numpy.min(duration))
    result.append(numpy.max(duration))
    return result

In [ ]:
#reshaping  loaded rasters to help pixel wise monitoring
pixel_val_i=[]
for i in perc_anml_val_3:
    pixel_val_i_10=[]
    for j in range(0,6446):
        pixel_val_i_10.append(i[j])
    pixel_val_i.append(pixel_val_i_10)

In [ ]:
#pixel wise monitoring
valid_frequency_count=[]
valid_mode_onset_month=[]
valid_mode_cessation_month=[]
valid_mean_duration=[]
valid_min_duration=[]
valid_max_duration=[]
for i in range(0,6446):
    pixel_val_i_calc=[]
    for j in pixel_val_i:
        pixel_val_i_calc.append(j[i])
    pixel_val_i_calc_rle=rle_2(pixel_val_i_calc)
    print("Results for pixel "+ str(i+1))
    result=monitoring(pixel_val_i_calc_rle)
    valid_frequency_count.append(result[0])
    valid_mode_onset_month.append(result[1])
    valid_mode_cessation_month.append(result[2])
    valid_mean_duration.append(result[3])
    valid_min_duration.append(result[4])
    valid_max_duration.append(result[5])

In [ ]:
#ceating lists to help in final results raster creation
frequency_values=[-9999]*12649
mode_onset_month_values=[-9999]*12649
mode_cessation_month_values=[-9999]*12649
mean_duration_values=[-9999]*12649
min_duration_values=[-9999]*12649
max_duration_values=[-9999]*12649
j=0
for i in mask_in:
    frequency_values[i]=valid_frequency_count[j]
    mode_onset_month_values[i]=valid_mode_onset_month[j]
    mode_cessation_month_values[i]=valid_mode_cessation_month[j]
    mean_duration_values[i]=valid_mean_duration[j]
    min_duration_values[i]=valid_min_duration[j]
    max_duration_values[i]=valid_max_duration[j]
    j=j+1

In [ ]:
# Frequncy Image
frequency_values=numpy.array(frequency_values)
frequencyMatrix=frequency_values.reshape(rows,col)
raster_frequency=driver.Create("Frequency_of_Drought.tif",col,rows,1,gdal.GDT_Float32)
raster_frequency.SetGeoTransform(img1.GetGeoTransform())
raster_frequency.SetProjection(img1.GetProjection())
raster_frequency.GetRasterBand(1).WriteArray(frequencyMatrix)
del raster_frequency

In [ ]:
# Mode Onset Month Image
mode_onset_month_values=numpy.array(mode_onset_month_values)
mode_monthMatrix=mode_onset_month_values.reshape(rows,col)
raster_mode_month=driver.Create("Mode_of_Onset_Month_of_Drought.tif",col,rows,1,gdal.GDT_Float32)
raster_mode_month.SetGeoTransform(img1.GetGeoTransform())
raster_mode_month.SetProjection(img1.GetProjection())
raster_mode_month.GetRasterBand(1).WriteArray(mode_monthMatrix)
del raster_mode_month

In [ ]:
# Mode Cessation Month Image
mode_cessation_month_values=numpy.array(mode_cessation_month_values)
mode_month_endMatrix=mode_cessation_month_values.reshape(rows,col)
raster_mode_month_end=driver.Create("Mode_of_Cessation_Month_of_Drought_End.tif",col,rows,1,gdal.GDT_Float32)
raster_mode_month_end.SetGeoTransform(img1.GetGeoTransform())
raster_mode_month_end.SetProjection(img1.GetProjection())
raster_mode_month_end.GetRasterBand(1).WriteArray(mode_month_endMatrix)
del raster_mode_month_end

In [ ]:
# Mean Duration Image
mean_duration_values=numpy.array(mean_duration_values)
mean_durationMatrix=mean_duration_values.reshape(rows,col)
raster_mean_duration=driver.Create("Mean_Duration.tif",col,rows,1,gdal.GDT_Float32)
raster_mean_duration.SetGeoTransform(img1.GetGeoTransform())
raster_mean_duration.SetProjection(img1.GetProjection())
raster_mean_duration.GetRasterBand(1).WriteArray(mean_durationMatrix)
del raster_mean_duration

In [ ]:
# Minimum Duration Image
min_duration_values=numpy.array(min_duration_values)
min_durationMatrix=min_duration_values.reshape(rows,col)
raster_min_duration=driver.Create("Minimum_Duration.tif",col,rows,1,gdal.GDT_Float32)
raster_min_duration.SetGeoTransform(img1.GetGeoTransform())
raster_min_duration.SetProjection(img1.GetProjection())
raster_min_duration.GetRasterBand(1).WriteArray(min_durationMatrix)
del raster_min_duration

In [ ]:
# Max Duration Image
max_duration_values=numpy.array(max_duration_values)
max_durationMatrix=max_duration_values.reshape(rows,col)
raster_max_duration=driver.Create("Maximum_Duration.tif",col,rows,1,gdal.GDT_Float32)
raster_max_duration.SetGeoTransform(img1.GetGeoTransform())
raster_max_duration.SetProjection(img1.GetProjection())
raster_max_duration.GetRasterBand(1).WriteArray(max_durationMatrix)
del raster_max_duration

# Monitoring Phase 2- Intensity(Average and Maximum)

# This process is made more efficient in SPImonitoring.ipynb 

In [ ]:
#reload rasters for intensity calculation
sdt_anml_val_4=[]
for i in range (0,len(sdt_anml)):
    img=gdal.Open(sdt_anml[i])
    val=img.GetRasterBand(1).ReadAsArray().flatten()
    val=masking_3(val)
    sdt_anml_val_4.append(val)
print(len(sdt_anml_val_4))

In [ ]:
sdt_anml_val_5=[]
for i in range(0,len(mask_in)):
    tempo_val=[]
    for j in sdt_anml_val_4:
        tempo_val.append(j[i])
    tempo_val=classification_1(tempo_val)
    sdt_anml_val_5.append(tempo_val)

In [ ]:
def monitoring_2(abc):
    result=[]
    freq=0
    month_count=0
    start_month=[]
    end_month=[]
    for k in range(0,len(abc),2):
        if abc[k]==0 and abc[k+1]>=3:
            freq+=1
            start_month.append(month_count)
            month_count=month_count+abc[k+1]
            end_month.append(month_count)
        else:
            month_count=month_count+abc[k+1]
    result.append(freq)
    result.append(start_month)
    result.append(end_month)
    return result

In [ ]:
perc_anml_val_5_rle=[]
for i in perc_anml_val_5:
    perc_anml_val_5_rle.append(rle_2(i))

In [ ]:
result_check=[]
for i in perc_anml_val_5_rle:
    result_check.append(monitoring_2(i))

In [ ]:
perc_anml_val_6=[]
for i in range(0,len(mask_in)):
    pixel_val=[]
    for j in perc_anml_val_4:
        pixel_val.append(j[i])
    perc_anml_val_6.append(pixel_val)

In [ ]:
avg_intensity_values=[]
max_intensity_values=[]
for i in range(0,len(result_check)):
    intensity_values=[]
    for j in range(0,result_check[i][0]):
        a=result_check[i][1][j]
        b=result_check[i][2][j]
        int_values=statistics.mean(perc_anml_val_6[i][a:b])
        intensity_values.append(int_values)
    avg_intensity_values.append(statistics.mean(intensity_values))
    max_intensity_values.append(numpy.min(intensity_values))

In [ ]:
average_intensity_values=[-9999]*12649
maximum_intensity_values=[-9999]*12649
j=0
for i in mask_in:
    average_intensity_values[i]=avg_intensity_values[j]
    maximum_intensity_values[i]=max_intensity_values[j]
    j=j+1

In [ ]:
# Average Intensity Image
average_intensity_values=numpy.array(average_intensity_values)
average_intensityMatrix=average_intensity_values.reshape(rows,col)
raster_average_intensity=driver.Create("Average_Intensity.tif",col,rows,1,gdal.GDT_Float32)
raster_average_intensity.SetGeoTransform(img1.GetGeoTransform())
raster_average_intensity.SetProjection(img1.GetProjection())
raster_average_intensity.GetRasterBand(1).WriteArray(average_intensityMatrix)
del raster_average_intensity

In [ ]:
# Maximum Intensity Image
maximum_intensity_values=numpy.array(maximum_intensity_values)
max_intensityMatrix=maximum_intensity_values.reshape(rows,col)
raster_max_intensity=driver.Create("Maximum_Intensity.tif",col,rows,1,gdal.GDT_Float32)
raster_max_intensity.SetGeoTransform(img1.GetGeoTransform())
raster_max_intensity.SetProjection(img1.GetProjection())
raster_max_intensity.GetRasterBand(1).WriteArray(max_intensityMatrix)
del raster_max_intensity